In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [2]:
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [3]:
df = pd.read_csv("../data/stemmed_merged_kubu_01.csv") 
print(df.columns)

Index(['created_at', 'id_str', 'full_text', 'lang', 'location'], dtype='object')


In [4]:
df['full_text'] = df['full_text'].str.replace(r'(wk)+|\bsih\b|\bya\b', '', regex=True)

In [5]:
docs = df['full_text'].dropna().astype(str).tolist()

In [6]:
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embedding_model.to('cuda' if torch.cuda.is_available() else 'cpu')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [7]:
pca_model = PCA(n_components=25)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=15,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True,
)


In [8]:
topic_model_hdbscan = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    calculate_probabilities=True,
    verbose=True
)

topics_hdbscan, probs = topic_model_hdbscan.fit_transform(docs)

topic_model_hdbscan.reduce_topics(docs, nr_topics=15)


2025-05-22 10:15:19,555 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/934 [00:00<?, ?it/s]

2025-05-22 10:15:24,756 - BERTopic - Embedding - Completed ✓
2025-05-22 10:15:24,756 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-22 10:15:39,738 - BERTopic - Dimensionality - Completed ✓
2025-05-22 10:15:39,740 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-22 10:16:34,284 - BERTopic - Cluster - Completed ✓
2025-05-22 10:16:34,289 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-22 10:16:34,516 - BERTopic - Representation - Completed ✓
2025-05-22 10:16:34,842 - BERTopic - Topic reduction - Reducing number of topics
2025-05-22 10:16:34,856 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-22 10:16:34,982 - BERTopic - Representation - Completed ✓
2025-05-22 10:16:34,984 - BERTopic - Topic reduction - Reduced number of topics from 206 to 15


In [9]:
# Topic summary table
topic_info = topic_model_hdbscan.get_topic_info()
print(topic_info)

# Interactive visualizations
topic_model_hdbscan.visualize_topics()
topic_model_hdbscan.visualize_barchart(top_n_topics=15)
topic_model_hdbscan.visualize_heatmap()


    Topic  Count                                 Name  \
0      -1   9870              -1_01_anies_dukung_imin   
1       0  10382              0_01_dukung_calon_pilih   
2       1   2979  1_anies_baswedan_indonesia_presiden   
3       2   1680        2_debat_calon_presiden_kritik   
4       3   1544  3_politik_partai_demokrasi_demokrat   
5       4   1514               4_cak_imin_anies_kayak   
6       5    559            5_video_videotron_01_film   
7       6    433        6_twitter_media_berita_sosial   
8       7    358                7_pop_korea_02_konsep   
9       8    233              8_program_anies_01_misi   
10      9    152      9_palestina_israel_merdeka_bela   
11     10     52          10_respons_reply_respon_cak   
12     11     51          11_viral_viralkan_ham_virus   
13     12     33         12_data_luhut_cherry_tambang   
14     13     21             13_estate_food_tani_blak   

                                       Representation  \
0   [01, anies, dukung, imin, 

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_01_dukung_calon, 1_anies_baswedan_indonesia,
                    2_debat_calon_presiden, 3_politik_partai_demokrasi,
                    4_cak_imin_anies, 5_video_videotron_01, 6_twitter_media_berita,
                    7_pop_korea_02, 8_program_anies_01, 9_palestina_israel_merdeka,
                    10_respons_reply_respon, 11_viral_viralkan_ham,
                    12_data_luhut_cherry, 13_estate_food_tani],
              'xaxis': 'x',
              'y': [0_01_dukung_calon, 1_anies_baswedan_indonesia,
                    2_debat_calon_presiden, 3_politik_partai_demokrasi,
                    4_cak_imin_anies, 5_video_videotron_01, 6_twitter_media_berita,
                    7_pop_korea_02, 8_program_anies_01, 9_palestina_israel_merdeka,
                    10_respons_reply_respon, 11_viral_viralkan_ham,
                    12_data_luhut_cherry, 13_estate_food_tani],
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAAAA8D8WH80+jgTiP41pXejZYe' ... 'iCKlPfPxjgC/WjEuE/AgAAAAAA8D8='),
                    'dtype': 'f8',
                    'shape': '14, 14'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [10]:
topic_info = topic_model_hdbscan.get_topic_info()
display(topic_info.head())        

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9870,-1_01_anies_dukung_imin,"[01, anies, dukung, imin, cak, pilih, calon, o...",[presiden terima kasih tingkat standar kampany...
1,0,10382,0_01_dukung_calon_pilih,"[01, dukung, calon, pilih, pasang, orang, mena...",[asli haru banget rakyat indonesia jalan jis d...
2,1,2979,1_anies_baswedan_indonesia_presiden,"[anies, baswedan, indonesia, presiden, rasyid,...",[semangat indonesia milik pimpin rakyat anies ...
3,2,1680,2_debat_calon_presiden_kritik,"[debat, calon, presiden, kritik, cak, imin, an...",[bukti debat calon wakil presiden cak imin huj...
4,3,1544,3_politik_partai_demokrasi_demokrat,"[politik, partai, demokrasi, demokrat, anies, ...",[ketua partai pegang partai demokrasi indonesi...


In [11]:
for tid in topic_info['Topic'][:15]:   # first 10 topics
    print(f"\nTopic {tid}")
    print(topic_model_hdbscan.get_topic(tid)[:15])  # top-15 words



Topic -1
[('01', np.float64(0.044657594563075206)), ('anies', np.float64(0.03835662477102073)), ('dukung', np.float64(0.030574327089240915)), ('imin', np.float64(0.03004313859230322)), ('cak', np.float64(0.02951789471657971)), ('pilih', np.float64(0.02753467247489603)), ('calon', np.float64(0.019261171367582337)), ('orang', np.float64(0.01917101292149584)), ('amin', np.float64(0.018896133234297143)), ('presiden', np.float64(0.018584025074659095))]

Topic 0
[('01', np.float64(0.07526216246939593)), ('dukung', np.float64(0.03685685200245356)), ('calon', np.float64(0.03114196267161134)), ('pilih', np.float64(0.030023127656557364)), ('pasang', np.float64(0.023760180007082594)), ('orang', np.float64(0.023725092252510106)), ('menang', np.float64(0.023707664677412584)), ('presiden', np.float64(0.0216531265469141)), ('kayak', np.float64(0.01854468703828625)), ('banget', np.float64(0.017949223351798608))]

Topic 1
[('anies', np.float64(0.07736698961806611)), ('baswedan', np.float64(0.051277017

In [12]:
topic_model_hdbscan.visualize_barchart(top_n_topics=15)
topic_model_hdbscan.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, '01 | dukung | calon | pilih | pasang', 10382],
                                   [1, 'anies | baswedan | indonesia | presiden | rasyid', 2979],
                                   [2, 'debat | calon | presiden | kritik | cak', 1680],
                                   [3, 'politik | partai | demokrasi | demokrat | anies', 1544],
                                   [4, 'cak | imin | anies | kayak | baswedan', 1514],
                                   [5, 'video | videotron | 01 | film | lihat', 559],
                                   [6, 'twitter | media | berita | sosial | 01', 433],
                                   [7, 'pop | korea | 02 | konsep | 01', 358],
                                   [8, 'program | anies | 01 | misi | visi', 233],
                                   [9, 'palestina | israel | merdeka | bela | aksi', 152],
                                   [10, 'respons | reply | respon | cak | imin', 52],
                                   [11, 'viral | viralkan | ham | virus | anies', 51],
                                   [12, 'data | luhut | cherry | tambang | cak', 33],
                                   [13, 'estate | food | tani | blak | imin', 21]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'jiijC5AGCAbqBS8CsQFmAekAmAA0ADMAIQAVAA==', 'dtype': 'i2'},
                         'sizemode': 'area',
                         'sizeref': 6.48875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': '9xmgQYKJ7L/Vz4i/1SfJv6d+aL+52KRBJE+dQRbNoUEIs5tB0siwvxpfLr82qaVBkWSmvz15BsA=',
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': 'dlP3wMq060DKIwVBdb0AQUGTw0Db2fvAaHDswOGlBcEMQADBRPHgQImRvkBzNArBJ2ztQOZI3EA=',
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(-2.4163222),
                                'y': np.float32(-0.18202591),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(10.698722),
                                'xshift': 10,
                                'y': np.float32(9.569424)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(10.698722),
                           'x1': np.float32(10.698722),
                           'y0': np.float32(-9.9334755),
                           'y1': np.float32(9.569424)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(-2.4163222),
                           'x1': np.float32(23.813766),
                           'y0': np.float32(-0.18202591),
                           'y1': np.float32(-0.18202591)}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red',
                                                                  '#B0BEC5',
                                                       

In [13]:
# df['created_at'] = pd.to_datetime(df['created_at'])

In [14]:
# topics_over_time = topic_model_hdbscan.topics_over_time(
#     docs=docs,
#     topics=topics_hdbscan,
#     timestamps=df['created_at'],
#     nr_bins=20,  # Adjust to control time granularity
#     evolution_tuning=True,
#     global_tuning=True
# )

In [15]:
# topic_model_hdbscan.visualize_topics_over_time(
#     topics_over_time, 
#     top_n_topics=10  # Number of top topics to show
# )


In [16]:
# Save results to CSV
df_topics = pd.DataFrame({"Document": docs, "Topic": topics_hdbscan})
df_topics.to_csv("topic_assignments_hdbscan.csv", index=False)

# Save model
# topic_model.save("bertopic_model_hdbscan")


In [13]:
from sklearn.cluster import KMeans

num_topics = 15
kmeans_model = KMeans(n_clusters=num_topics, random_state=42)

topic_model_kmeans = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=kmeans_model,
    calculate_probabilities=False,
    verbose=True
)

topics_kmeans, _ = topic_model_kmeans.fit_transform(docs)

topic_model_kmeans.visualize_barchart(top_n_topics=15)

2025-05-22 12:04:33,346 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/934 [00:00<?, ?it/s]

2025-05-22 12:04:38,509 - BERTopic - Embedding - Completed ✓
2025-05-22 12:04:38,510 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-22 12:04:40,784 - BERTopic - Dimensionality - Completed ✓
2025-05-22 12:04:40,786 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-22 12:04:40,939 - BERTopic - Cluster - Completed ✓
2025-05-22 12:04:40,945 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-22 12:04:41,114 - BERTopic - Representation - Completed ✓


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02891636204398661, 0.029205198271782782,
                    0.037948476104567454, 0.05080941147309609, 0.0771922877941425],
              'xaxis': 'x',
              'y': [pasang  , pilih  , orang  , dukung  , 01  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.027421154467304733, 0.030602561745866348,
                    0.035872245111182206, 0.06138137985566899, 0.11554277495623003],
              'xaxis': 'x2',
              'y': [buzzer  , pasang  , pilih  , dukung  , 01  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04209595079374513, 0.04508020447036836,
                    0.047310535139710216, 0.056487032786511716,
                    0.07949788701704243],
              'xaxis': 'x3',
              'y': [rasyid  , cak  , imin  , baswedan  , anies  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02131614622349928, 0.02399918841722143, 0.03466486855430646,
                    0.0531542971346682, 0.19632775559089122],
              'xaxis': 'x4',
              'y': [bang  , amin  , pilih  , dukung  , 01  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.026318409920251522, 0.02770067127609489, 0.0281829790017286,
                    0.03098110087946016, 0.0475807452926366],
              'xaxis': 'x5',
              'y': [pasang  , orang  , kampanye  , dukung  , 01  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.037319462726674676, 0.0394670069279208,
                    0.059978201999192016, 0.06988712533422983, 0.10413526080627769],
              'xaxis': 'x6',
              'y': [baswedan  , calon  , pilih  , anies  , presiden  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04011888289893154, 0.04415990536663801, 0.04427302822949799,
                    0.05464400117949251, 0.06865704159758221],
              'xaxis': 'x7',
              'y': [baswedan  , cak  , imin  , indonesia  , anies  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0222361915170638, 0.02775800344796687, 0.07473900577980004,
                    0.18267903065599034, 0.18944443315034068],
              'xaxis': 'x8',
              'y': [baswedan  , banget  , anies  , imin  , cak  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03533645135391245, 0.041488034825448314,
                    0.05355932605394615, 0.06369332890736079, 0.09057401566866607],
              'xaxis': 'x9',
              'y': [twitter  , dukung  , video  , 01  , kayak  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03759229914871937, 0.0501473531624071, 0.058450679149207645,
                    0.06150959975096537, 0.07119778928761121],
              'xaxis': 'x10',
              'y': [agama  , pintar  , 01  , indonesia  , suara  ],
              'yaxis': 'y10'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03563521312526459, 0.048

In [14]:
# Topic summary table
topic_info = topic_model_kmeans.get_topic_info()
print(topic_info)

# Interactive visualizations
topic_model_kmeans.visualize_topics()
topic_model_kmeans.visualize_barchart(top_n_topics=15)
topic_model_kmeans.visualize_heatmap()


    Topic  Count                            Name  \
0       0   3249         0_01_dukung_orang_pilih   
1       1   3151        1_01_dukung_pilih_pasang   
2       2   2856       2_anies_baswedan_imin_cak   
3       3   2752          3_01_dukung_pilih_amin   
4       4   2696      4_01_dukung_kampanye_orang   
5       5   2321    5_presiden_anies_pilih_calon   
6       6   2203      6_anies_indonesia_imin_cak   
7       7   2084         7_cak_imin_anies_banget   
8       8   1801         8_kayak_01_video_dukung   
9       9   1650     9_suara_indonesia_01_pintar   
10     10   1555      10_presiden_calon_01_wakil   
11     11   1199        11_menang_01_survei_gara   
12     12    885  12_politik_demokrasi_partai_01   
13     13    748      13_debat_calon_01_presiden   
14     14    711      14_debat_cak_imin_presiden   

                                       Representation  \
0   [01, dukung, orang, pilih, pasang, banget, tem...   
1   [01, dukung, pilih, pasang, buzzer, banget, ca...

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_01_dukung_orang, 1_01_dukung_pilih, 2_anies_baswedan_imin,
                    3_01_dukung_pilih, 4_01_dukung_kampanye,
                    5_presiden_anies_pilih, 6_anies_indonesia_imin,
                    7_cak_imin_anies, 8_kayak_01_video, 9_suara_indonesia_01,
                    10_presiden_calon_01, 11_menang_01_survei,
                    12_politik_demokrasi_partai, 13_debat_calon_01,
                    14_debat_cak_imin],
              'xaxis': 'x',
              'y': [0_01_dukung_orang, 1_01_dukung_pilih, 2_anies_baswedan_imin,
                    3_01_dukung_pilih, 4_01_dukung_kampanye,
                    5_presiden_anies_pilih, 6_anies_indonesia_imin,
                    7_cak_imin_anies, 8_kayak_01_video, 9_suara_indonesia_01,
                    10_presiden_calon_01, 11_menang_01_survei,
                    12_politik_demokrasi_partai, 13_debat_calon_01,
                    14_debat_cak_imin],
              'yaxis': 'y',
              'z': {'bdata': ('BACAP1CHdD8IpSE/VOdPP6K4cT+TWw' ... 'UNP/lHHj8GFuA+yjsXP0rbVD/+/38/'),
                    'dtype': 'f4',
                    'shape': '15, 15'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [15]:
topic_info = topic_model_kmeans.get_topic_info()
display(topic_info.head())        

,Topic,Count,Name,Representation,Representative_Docs
0,0,3249,0_01_dukung_orang_pilih,"[01, dukung, orang, pilih, pasang, banget, tem...","[orang malu pilih 01, orang pilih 01 dukung am..."
1,1,3151,1_01_dukung_pilih_pasang,"[01, dukung, pilih, pasang, buzzer, banget, ca...","[sopan banget dukung amin dukung 01, kasihan p..."
2,2,2856,2_anies_baswedan_imin_cak,"[anies, baswedan, imin, cak, rasyid, ayah, ami...",[tim nasional timnas menang anies baswedan muh...
3,3,2752,3_01_dukung_pilih_amin,"[01, dukung, pilih, amin, bang, buzzer, coblos...","[ dukung 01 harap, asli dukung 01 amin, dukung..."
4,4,2696,4_01_dukung_kampanye_orang,"[01, dukung, kampanye, orang, pasang, pilih, k...",[ribut pasang calon a b c omong bicara data co...


In [16]:
for tid in topic_info['Topic'][:15]:   # first 10 topics
    print(f"\nTopic {tid}")
    print(topic_model_kmeans.get_topic(tid)[:15])  # top-15 words



Topic 0
[('01', np.float64(0.0771922877941425)), ('dukung', np.float64(0.05080941147309609)), ('orang', np.float64(0.037948476104567454)), ('pilih', np.float64(0.029205198271782782)), ('pasang', np.float64(0.02891636204398661)), ('banget', np.float64(0.024384181851401388)), ('teman', np.float64(0.02398189712478682)), ('calon', np.float64(0.021505741597132855)), ('buzzer', np.float64(0.01612408369899807)), ('bilang', np.float64(0.01574043530658533))]

Topic 1
[('01', np.float64(0.11554277495623003)), ('dukung', np.float64(0.06138137985566899)), ('pilih', np.float64(0.035872245111182206)), ('pasang', np.float64(0.030602561745866348)), ('buzzer', np.float64(0.027421154467304733)), ('banget', np.float64(0.023402408122674034)), ('calon', np.float64(0.02219665300763565)), ('bang', np.float64(0.019595020859839805)), ('amin', np.float64(0.01850916308741227)), ('kubu', np.float64(0.017474347201589323))]

Topic 2
[('anies', np.float64(0.07949788701704243)), ('baswedan', np.float64(0.05648703278

In [17]:
topic_model_kmeans.visualize_barchart(top_n_topics=15)
topic_model_kmeans.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, '01 | dukung | orang | pilih | pasang', 3249],
                                   [1, '01 | dukung | pilih | pasang | buzzer', 3151],
                                   [2, 'anies | baswedan | imin | cak | rasyid', 2856],
                                   [3, '01 | dukung | pilih | amin | bang', 2752],
                                   [4, '01 | dukung | kampanye | orang | pasang', 2696],
                                   [5, 'presiden | anies | pilih | calon | baswedan', 2321],
                                   [6, 'anies | indonesia | imin | cak | baswedan', 2203],
                                   [7, 'cak | imin | anies | banget | baswedan', 2084],
                                   [8, 'kayak | 01 | video | dukung | twitter', 1801],
                                   [9, 'suara | indonesia | 01 | pintar | agama', 1650],
                                   [10, 'presiden | calon | 01 | wakil | pilih', 1555],
                                   [11, 'menang | 01 | survei | gara | sukses', 1199],
                                   [12, 'politik | demokrasi | partai | 01 | demokrat', 885],
                                   [13, 'debat | calon | 01 | presiden | strategi', 748],
                                   [14, 'debat | cak | imin | presiden | anies', 711]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'sQxPDCgLwAqIChEJmwgkCAkHcgYTBq8EdQPsAscC', 'dtype': 'i2'},
                         'sizemode': 'area',
                         'sizeref': 2.030625,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': 'kSpnQX+Pa0HiS8zAtNdyQdeAWkFaLdzA9RfWwJ7zw8AIGXZBZ2BfQfteVEEEaG1BRXZWQe+VaEDVjl5A',
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': 'SRCRwO+1ecDwMyFBJhJjwJ/epcDZhBlBsIAcQXJaJUElpI3AiwaxwAb3usCmSZvAQ/HHwDQTZECOGm5A',
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(-7.9126167),
                                'y': np.float32(2.3496687),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(4.8878307),
                                'xshift': 10,
                                'y': np.float32(11.8847685)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(4.8878307),
                           'x1': np.float32(4.8878307),
                           'y0': np.float32(-7.185431),
                           'y1': np.float32(11.8847685)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(-7.9126167),
                           'x1': np.float32(17.688278),
                           'y0': np.float32(2.3496687),
                           'y1': np.float32(2.3496687)}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red',
                         

In [18]:
# Save results to CSV
df_topics = pd.DataFrame({"Document": docs, "Topic": topics_kmeans})
df_topics.to_csv("topic_assignments_kmeans.csv", index=False)

# Save model
# topic_model.save("bertopic_model_kmeans")

In [19]:
topics_over_time_kmeans = topic_model_kmeans.topics_over_time(
    docs=docs,
    topics=topics_hdbscan,
    timestamps=df['created_at'],
    nr_bins=20,
    evolution_tuning=True,
    global_tuning=True
)

topic_model_hdbscan.visualize_topics_over_time(topics_over_time_kmeans, top_n_topics=10)

0it [00:00, ?it/s]


IndexError: index (184) out of range

In [20]:
print("HDBSCAN Topics:", len(set(topics_hdbscan)) - (1 if -1 in topics_hdbscan else 0))
print("KMeans Topics:", len(set(topics_kmeans)))

# Outlier analysis
print("HDBSCAN Outliers:", topics_hdbscan.count(-1))  # KMeans won't have outliers

HDBSCAN Topics: 205
KMeans Topics: 15
HDBSCAN Outliers: 9870


In [21]:
import pandas as pd
from collections import Counter

# Count documents per topic
hdbscan_counts = Counter(topics_hdbscan)
kmeans_counts = Counter(topics_kmeans)

# Create DataFrame to compare
comparison_df = pd.DataFrame({
    "HDBSCAN": pd.Series(hdbscan_counts),
    "KMeans": pd.Series(kmeans_counts)
}).fillna(0).astype(int)

comparison_df.sort_index()


,HDBSCAN,KMeans
-1,9870,0
0,1015,3249
1,776,3151
2,568,2856
3,545,2752
...,...,...
200,16,0
201,16,0
202,16,0
203,15,0
